In [1]:
import numpy as np
import time
import random
import scipy
from sklearn.datasets import load_svmlight_file
from sklearn.datasets import fetch_mldata
from sklearn.linear_model import SGDClassifier

In [2]:
class DataLoader(object):
    
    def __init__(self, dataset_name):
        self.loadLIBSVMData(dataset_name)
        if dataset_name == "scene":
            self.num_class = 6
        elif dataset_name == "yeast":
            self.num_class = 14
        elif dataset_name == "siam":
            self.num_class = 22
        elif dataset_name == "topics":
            self.num_class = 101
        
        self.train_data_size = self.train_data.shape[0]
        self.test_data_size = self.test_data.shape[0]
        self.num_feature = self.train_data.shape[1]
        
        self.shuffle()
    
    def loadLIBSVMData(self, name):
        train_data, train_labels = load_svmlight_file('./datasets/{name}/{name}_train.svm'.format(name=name), multilabel=True)
        self.train_data = train_data.toarray()
        self.train_labels = train_labels
        test_data, test_labels = load_svmlight_file('./datasets/{name}/{name}_test.svm'.format(name=name), multilabel=True)
        self.test_data = test_data.toarray()
        self.test_labels = test_labels
    
    def shuffle(self):
        shuffle = np.random.permutation(self.train_data_size)
        self.train_data = self.train_data[shuffle]
        labels = []
        for i in shuffle:
            labels.append(self.train_labels[i])
        self.train_labels = labels
        
        shuffle = np.random.permutation(self.test_data_size)
        self.test_data = self.test_data[shuffle]
        labels = []
        for i in shuffle:
            labels.append(self.test_labels[i])
        self.test_labels = labels
        
    def trainGenerator(self, method):
        if method == 'PT1':
            for i in range(self.train_data_size):
                rand = random.randint(0, len(self.train_labels[i]) - 1)
                yield (self.train_data[i].reshape(1, -1), self.train_labels[i][rand])
        elif method == 'PT3':
            for i in range(self.train_data_size):
                res = 0
                for label in self.train_labels[i]:
                    res += 1 << int(label)
                yield (self.train_data[i].reshape(1, -1), res)
        elif method == 'PT5':
            for i in range(self.train_data_size):
                for label in self.train_labels[i]:
                    yield (self.train_data[i].reshape(1, -1), res)
    
    def testGenerator(self, method):
        if method == 'PT3':
            for i in range(self.test_data_size):
                res = 0
                for label in self.train_labels[i]:
                    res += 1 << int(label)
                yield (self.test_data[i].reshape(1, -1), res)
        else:
            for i in range(self.test_data_size):
                yield (self.test_data[i].reshape(1, -1), self.test_labels[i])

In [3]:
class OnlineClassification(object):
    
    def __init__(self, num_feature, learning_rate='constant', eta0=0.01):
        self.classifier = SGDClassifier(learning_rate=learning_rate, eta0=eta0, warm_start=True)
        
    def train(self, x, c):
        self.classifier.partial_fit(x, np.array([c]), [-1, 1])
        
    def test(self, x):
        return self.classifier.predict(x)

In [4]:
class Node(object):
    
    def __init__(self, num_feature, learning_rate='constant', eta0=0.01):
        self.left = None
        self.right = None
        self.parent = None
        self.max_label = 1
        self.max_label_count = 0
        self.n_all = 0
        self.m_all = 0
        self.C = 0
        self.l = {}
        self.n = {}
        self.m = {}
        self.model = OnlineClassification(num_feature, learning_rate=learning_rate, eta0=eta0)
    
    def reset(self):
        self.left = None
        self.right = None
        self.parent = None
        self.max_label = 1
        self.max_label_count = 0
        self.n_all = 0
        self.m_all = 0
        self.C = 0
        self.l.clear()
        self.n.clear()
        self.m.clear()

    def testModel(self, x):
        return self.model.test(x)
    
    def trainModel(self, x, c):
        self.model.train(x, c)
        
    def addClass(self, class_name):
        self.n[class_name] = 0
        self.m[class_name] = 0
        self.l[class_name] = 0
        
    def findExpectationAll(self):
        if self.n_all == 0:
            return 0
        else:
            return self.m_all / self.n_all
        
    def findExpectationOneClass(self, y):
        if self.n[y] == 0:
            return 0
        else:
            return self.m[y] / self.n[y]
    
    def judgeInTrain(self, y):
        #c == -1: left, c == 1: right
        return -1 if self.findExpectationAll() > self.findExpectationOneClass(y) else 1

In [5]:
class Tree(object):
    def __init__(self, T, data_loader, method='PT1', threshold=0.5, Rs=16, epoch=1, learning_rate='constant', eta0=0.01):
        self.data_loader = data_loader
        self.num_feature = data_loader.num_feature
        self.method = method
        self.threshold = threshold
        self.eta0 = eta0;
        self.learning_rate = learning_rate
        self.epoch = epoch
        self.Rs = Rs
        self.T = T
        self.t = 1
        self.size = 0
        self.root = self.generateNode()
        
    def generateNode(self):
        node = Node(self.num_feature, self.learning_rate, self.eta0)
        self.size = self.size + 1
        return node
        
    def split(self, node):
        self.t = self.t + 1
        left = self.generateNode()
        right = self.generateNode()
        
        node.left = left
        left.parent = node
        node.right = right
        right.parent = node
        
    def swap(self, node):
        cur = self.root
        while cur.left != None:
            cur = cur.left if cur.left.C < cur.right.C else cur.right
        
        parent = cur.parent
        grandpa = parent.parent
        sib = parent.left if parent.left == cur else parent.right
        if parent == grandpa.left:
            grandpa.left = sib
        else:
            grandpa.right = sib
        sib.parent = grandpa
        
        self.updateC(sib)
        cur.reset()
        parent.reset()
        node.left = cur
        cur.parent = node
        node.right = parent
        parent.parent = node
        
    def updateC(self, node):
        while node != self.root and node.parent.C != node.C:
            node = node.parent
            node.C = min(node.left.C, node.right.C)
    
    def train(self):
        start = time.time()
        print('Start training.......')
        for i in range(self.epoch):
            train_generator = self.data_loader.trainGenerator(self.method)
            for sample in train_generator:
                self.fitOne(sample)
            acc, pre, rec = self.test()
            print('epoch %d: >>>>>>>> acc=%.3f, pre=%.3f, rec=%.3f' % (i, acc, pre, rec))
            print('left: %d, right: %d' % (self.root.left.n_all, self.root.right.n_all))
        end = time.time()
        print('time used: %d s' % (end - start))
    
    def test(self):
        test_generator = self.data_loader.testGenerator(self.method)
        acc_list = []
        pre_list = []
        rec_list = []
        for sample in test_generator:
            x, y = sample
            y_hat = self.predict(x)
            acc, pre, rec = self.score(y, y_hat)
            acc_list.append(acc)
            pre_list.append(pre)
            rec_list.append(rec)
        
        return np.mean(acc_list), np.mean(pre_list), np.mean(rec_list)
    
    def score(self, y, y_hat):
        # used for debug
#         print (y, y_hat)
        if self.method == 'PT3':
            res = 0
            for i in y_hat:
                res = res | i
            intersect = bin(y & res).count('1')
            union = bin(y | res).count('1')
#             print(y, y_hat, intersect, union)
            return intersect / union, intersect / bin(res).count('1'), intersect / bin(y).count('1')
        else:
            y = set(y)
            y_hat = set(y_hat)
            intersect = len(y.intersection(y_hat))
            union = len(y.union(y_hat))
            return intersect / union, intersect / len(y_hat), intersect / len(y)
    
    def fitOne(self, xy):
        x, y = xy
        node = self.root
        #register if y is new in this node
        while node != None:
            not_registered = node.l.get(y) == None
            if not_registered:
                node.addClass(y)
            
            node.l[y] += 1
            
            if node.l[y] > node.max_label_count:
                node.max_label = y
                node.max_label_count = node.l[y]

            #give birth or swap in a leaf node if num_class >= 2 or 
            if node.left == None and len(node.n) > 1:
                if self.t < self.T or node.C - node.l[node.max_label] > self.Rs * (self.root.C + 1):
                    if self.t < self.T:
                        #give birth
                        self.split(node)
                    else:
                        #swap
                        self.swap(node)
                    node.left.C = node.C // 2
                    node.right.C = node.C - node.left.C
                    node.left.max_label = node.max_label
                    node.right.max_label = node.max_label
                    self.updateC(node.left)

            #train if node is not leaf
            if node.left != None:
                c = node.judgeInTrain(y)
                node.trainModel(x, c)
                c_hat = node.testModel(x)
                node.n_all += 1
                node.m_all += c_hat
                node.n[y] += 1
                node.m[y] += c_hat
                
                node = node.left if c_hat == -1 else node.right
            else:
                node.C += 1
                self.updateC(node)
                break

    def predict(self, x):
        node = self.root
        while node.left != None:
            node = node.left if node.testModel(x) == -1 else node.right
        max_frq = node.max_label_count
        res = []
        for label, frq in node.l.items():
            if frq > self.threshold * max_frq:
                res.append(label)
        return res

In [9]:
#build
dataset_name = 'scene'
data_loader = DataLoader(dataset_name)
K = data_loader.num_class
print('training data size: %d, test data size: %d, num of features: %d, num of classes: %d' % (data_loader.train_data_size, data_loader.test_data_size, data_loader.num_feature, data_loader.num_class))

training data size: 1211, test data size: 1196, num of features: 294, num of classes: 6


In [7]:
T = 4*K - 1
Rs = 256
learning_rate = 'optimal'
eta0 = 0.1
epoch = 5
method = 'PT3'
threshold = 0.5

LOM_tree = Tree(T, data_loader, method=method, threshold=threshold, epoch=epoch, Rs=Rs, learning_rate=learning_rate, eta0=eta0)

print('dataset_name=\'%s\', T=%d, Rs=%d, learning_rate=\'%s\', eta0=%.2f, epoch=%d' % (dataset_name, T, Rs, learning_rate,  eta0, epoch))
#train
LOM_tree.train()

dataset_name='siam', T=87, Rs=256, learning_rate='optimal', eta0=0.10, epoch=5
Start training.......


C:\Users\Whisper\Anaconda3\lib\site-packages\sklearn\linear_model\stochastic_gradient.py:128: FutureWarning: max_iter and tol parameters have been added in <class 'sklearn.linear_model.stochastic_gradient.SGDClassifier'> in 0.19. If both are left unset, they default to max_iter=5 and tol=None. If tol is not None, max_iter defaults to max_iter=1000. From 0.21, default max_iter will be 1000, and default tol will be 1e-3.
  "and default tol will be 1e-3." % type(self), FutureWarning)


KeyboardInterrupt: 

In [ ]:
# check balance
count = 0
node = LOM_tree.root
print(node.left.n_all, node.right.n_all)
while node != None:
    node = node.left
    count += 1
print(count)

count = 0
node = LOM_tree.root
while node != None:
    node = node.right
    count += 1
print(count)

In [ ]:
label = 11100010101
bin(5)